[![Binder](https://mybinder.org/badge_logo.svg)](https://nbviewer.org/github/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/07-DCT/block_DCT_compression.ipynb)

# Block-DCT (Discrete Cosine Transform) Image Compression

Compressing color images with PNG in the YCoCg/DCT domain. No chroma subsampling. Remember to run [JPEG.ipynb](https://github.com/Sistemas-Multimedia/Sistemas-Multimedia.github.io/blob/master/milestones/07-DCT/JPEG.ipynb) before!

## Parameters

In [ ]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import math
import os
import pylab
import cv2

!ln -sf ~/MRVC/src/image_3.py .
import image_3 as image
!ln -sf ~/MRVC/src/image_1.py .
import image_1 as component
!ln -sf ~/MRVC/src/block_DCT.py .
import block_DCT
!ln -sf ~/MRVC/src/YCoCg.py .
import YCoCg as color_transform
#!ln -sf ~/MRVC/src/color_DCT.py .
#import color_DCT as color
#!ln -sf ~/MRVC/src/RGB.py .
#import RGB as color
!ln -sf ~/quantization/information.py .
import information
!ln -sf ~/quantization/distortion.py .
import distortion
!ln -sf ~/quantization/deadzone_quantizer.py .
import deadzone_quantizer as Q

In [ ]:
HOME = os.environ["HOME"]
#test_image = "../sequences/stockholm/"
test_image = HOME + "/MRVC/sequences/lena_color/"
#test_image = "../sequences/lena_bw/"

In [ ]:
block_y_side = block_x_side = 8 # Block-size used by JPEG
# block_y_side = block_x_side = 16

In [ ]:
N_components = 3

In [ ]:
entropy_estimator = "PNG"
# entropy_estimator = "entropy"
if entropy_estimator == "PNG":
    def compute_BPP(_image, filename_prefix):
        BPP = image.write(_image, filename_prefix, 0)*8/_image.size
        return BPP
else:
    def compute_BPP(_image, filename_prefix=''):
        entropy = information.entropy(_image.flatten().astype(np.int16))
        return entropy

## Testing `block_DCT.analyze_block()` and `block_DCT.synthesize_block()`

Let's see how the DCT concentrates the energy of the signal in a few coefficients. These methods compute the forward and the backward transforms of the input block (the input is not divided into blocks).

In [ ]:
#a = np.random.randint(low=0, high=100, size=(4,4,3))
a = np.full(shape=(5, 4, 3), fill_value=10, dtype=np.int16) + np.random.randint(low=-5, high=5, size=(5, 4, 3))
# 5 is the number of rows
# 4 is the number of columns
# 3 is the number of (for example, RGB) channels

In [ ]:
a

In [ ]:
np.average(a)

In [ ]:
image.show(image.normalize(a))

In [ ]:
b = block_DCT.analyze_block(a)

In [ ]:
b

In [ ]:
b.astype(np.int16)

In [ ]:
image.show(image.normalize(b))

In [ ]:
c = block_DCT.synthesize_block(b)

In [ ]:
c.astype(np.int16)

In [ ]:
image.show(image.normalize(c))

In [ ]:
(a == c.astype(np.int32)).all()

## Testing `block_DCT.analyze_image()` and `block_DCT.synthesize_image()`
Now we apply the block transform to an image that previously has been divided into blocks.

In [ ]:
img = image.read(test_image, 0)
image.show(img, title="Original")

In [ ]:
DCT_img = block_DCT.analyze_image(img, block_y_side, block_x_side)
#DCT_img = block_DCT.analyze_image(img, 2, 2)

In [ ]:
image.show(image.normalize(DCT_img), f"{block_y_side}x{block_x_side}-DCT domain of {test_image}")

In [ ]:
image.show(image.normalize(DCT_img[:64, :64]), "detail [0:64, 0:64]")

Again, as it can be seen, most of the energy of each block has been concentrated in the low-pass frequency component (DC component).

### Reconstruction and error

In [ ]:
recons_img = block_DCT.synthesize_image(DCT_img, block_y_side, block_x_side)

In [ ]:
image.show(recons_img.astype(np.uint8), "Reconstructed image")

In [ ]:
error = img - recons_img

In [ ]:
image.show(image.normalize(error), "DCT floating-point error")
#image.show(error, "DCT floating-point error")

This error es generated by the truncation of the floating point coefficients (remember that we work with 16 bits integers) after the analysis, and also by the truncation of the floating point pixels after the synthesis. 

## Switching between blocks and subbands

The coefficients of all DCT-blocks can be reorganized in subbands. A subband with coordinates (X, Y) is the 2D arragement of the coefficients that are in the coordinates (X, Y) of each block. The representation in subbands increases the spatial correlation between the coefficients (which also provides an improved visual comprehension of the content of the coefficients).

In [ ]:
img = image.read(test_image)
DCT_blocks = block_DCT.analyze_image(img, block_y_side, block_x_side)
DCT_subbands = block_DCT.get_subbands(DCT_blocks, block_y_side, block_x_side)

In [ ]:
image.show(image.normalize(DCT_subbands), f"Subbands of the {block_y_side}x{block_x_side} DCT domain")

In [ ]:
print(f"We have {block_y_side}x{block_x_side} subbands of {int(img.shape[0]/block_y_side)}x{int(img.shape[1]/block_x_side)} coefficients (each one)")

The inverse process which reorder the coefficients into subbands is completely reversible (obviously).

In [ ]:
_ = block_DCT.get_blocks(DCT_subbands, block_y_side, block_x_side)
(_ == DCT_blocks).all()

And, as it can be seen, the 2D correlation is higher in the low spatial frequencies (left up corner) than in the high frequencies (right down corner).

In [ ]:
blocks_in_y = img.shape[0]//block_y_side
blocks_in_x = img.shape[1]//block_x_side
image.show(image.normalize(DCT_subbands[:blocks_in_y, :blocks_in_x]), f"Subband (0, 0) ({block_y_side}x{block_x_side} DCT)")

Subband (0,0) contains the low frequencies of the image.

In [ ]:
image.show(image.normalize(DCT_subbands[:blocks_in_y, blocks_in_x:2*blocks_in_x]), f"Subband (0, 1) ({block_y_side}x{block_x_side} DCT)")

The subband (0, 1) represents the slowest changes of the image in the horizontal direction.

In [ ]:
image.show(image.normalize(DCT_subbands[blocks_in_y:2*blocks_in_y, :blocks_in_x]), f"Subband (1, 0) ({block_y_side}x{block_x_side} DCT)")

The subband (1, 0) represents the slowest changes of the image in the vertical domain.

In [ ]:
image.show(image.normalize(DCT_subbands[blocks_in_y:2*blocks_in_y, blocks_in_x:2*blocks_in_x]), f"Subband (1, 1) ({block_y_side}x{block_x_side} DCT)")

The subband (1, 1) represents slowest changes in the diagonal (left up corner to right down corner) of the image.

## Subband-components information

In [ ]:
img = image.read(test_image, 0)
#YUV_img = color_transform.from_RGB(img.astype(np.int16) - 128) # -128 decreases maximum value of the DC coefficients
###############################################################
# This reduces the energy (not the entropy) of the            #
# coefficients compared to the previous option.               #
# However, the averages should be encoded to                  #
# reconstruct the image.                                      #
YUV_img = color_transform.from_RGB(img.astype(np.int16))      #
YUV_img[...,0] -= np.average(YUV_img[...,0]).astype(np.int16) #
YUV_img[...,1] -= np.average(YUV_img[...,1]).astype(np.int16) #
YUV_img[...,2] -= np.average(YUV_img[...,2]).astype(np.int16) #
###############################################################
DCT_blocks = block_DCT.analyze_image(YUV_img, block_y_side, block_x_side)
DCT_subbands = block_DCT.get_subbands(DCT_blocks, block_y_side, block_x_side)
print("sorting subband-components by entropy")
print("subband component maximum mininum max-min average std-dev entropy        energy  avg-enegy")
accumulated_entropy = 0
blocks_in_y = img.shape[0]//block_y_side
blocks_in_x = img.shape[1]//block_x_side
list_of_subbands_components = []
for _y in range(block_y_side):
    for _x in range(block_x_side):
        for _c in range(N_components):
            subband = DCT_subbands[blocks_in_y*_y:blocks_in_y*(_y+1), blocks_in_x*_x:blocks_in_x*(_x+1), _c]
            entropy = information.entropy(subband.flatten().astype(np.int16))
            accumulated_entropy += entropy
            max = subband.max()
            min = subband.min()
            max_min = max - min
            avg = np.average(subband)
            dev = math.sqrt(np.var(subband))
            energy = information.energy(subband)
            avg_energy = energy/subband.size
            list_of_subbands_components.append((_y, _x, _c, max, min, max_min, avg, dev, entropy, energy, avg_energy))
            #print(f"{_y:2d} {_x:2d} {_c:9d} {max:7.1f} {min:7.1f} {max_min:7.1f} {avg:7.1f} {dev:7.1f} {entropy:7.1f} {energy:13.1f} {avg_energy:10.1f}")
sorted_list_of_subbands_components = sorted(list_of_subbands_components, key=lambda x: x[8])[::-1]
for _i in sorted_list_of_subbands_components:
    _y, _x, _c, max, min, max_min, avg, dev, entropy, energy, avg_energy = _i
    print(f"  {_y:2d} {_x:2d} {_c:9d} {max:7.1f} {min:7.1f} {max_min:7.1f} {avg:7.1f} {dev:7.1f} {entropy:7.1f} {energy:13.1f} {avg_energy:10.1f}")
avg_entropy = accumulated_entropy/(block_x_side*block_y_side*img.shape[2])
print("Average entropy in the DCT domain:", avg_entropy)
print("Entropy in the image domain:", information.entropy(img.flatten().astype(np.uint8)))

As it can be observed, the 8x8-DCT accumulates most of the energy (and information, for this reason the entropy is decreased) in the low-frequency subbands. Notice also the high correlation that exists between the entropy, the variance and the energy of the subbands.

## Lossless compression

In [ ]:
img = image.read(test_image)
YUV_img = color_transform.from_RGB(img.astype(np.int16) - 128) # -128 decreases maximum value of the DC coefficients
DCT_blocks = block_DCT.analyze_image(YUV_img, block_y_side, block_x_side)
DCT_subbands = block_DCT.get_subbands(DCT_blocks, block_y_side, block_x_side)

img = image.read(test_image, 0)
YUV_img = color_transform.from_RGB(img.astype(np.int16) - 128)
DCT_blocks = block_DCT.analyze_image(YUV_img, block_y_side, block_x_side)
DCT_subbands = block_DCT.get_subbands(DCT_blocks, block_y_side, block_x_side)
DCT_subbands = (DCT_subbands + 32768).astype(np.uint16)
output_len = image.write(DCT_subbands, "/tmp/lossless", 0)
print(f"output_length={output_len}")
_DCT_subbands = image.read("/tmp/lossless", 0)
_DCT_subbands = _DCT_subbands.astype(np.float32) - 32768
_DCT_blocks = block_DCT.get_blocks(_DCT_subbands, block_y_side, block_x_side)
_YUV_img = block_DCT.synthesize_image(_DCT_blocks, block_y_side, block_x_side)
_img = color_transform.to_RGB(_YUV_img.astype(np.int16)) + 128
image.show(_img)

## Lossy compression

### Quantization steps

Considering the previous dynamic range values for the YCoCg/8x8-DCT coefficients, this parameter should allow to use 8 bits/pixel images, if we are using PNG as an entropy codec. As it can be seen, we need 11 bits for representing the DC coefficients and after quantization, we should use only 8. Therefore, the minimum quantization step should be 1<<3 = 8. Notice that 11 - 8 = 3.

In [ ]:
Q_steps = [128, 64, 32, 16, 8]

## Testing `block_DCT.uniform_quantize()` and `block_DCT.uniform_dequantize()` (MOVER A test_block_DCT)
Quantization removes information but also increases the compression ratios of the stored images. These methods quantize all coefficients with the same quantization step.

In [ ]:
Q_step = 64
img = image.read(test_image, 0)
YUV_img = color_transform.from_RGB(img.astype(np.int16) - 128)
DCT_blocks = block_DCT.analyze_image(YUV_img, block_y_side, block_x_side)
DCT_subbands = block_DCT.get_subbands(DCT_blocks, block_y_side, block_x_side)
DCT_subbands_k = block_DCT.uniform_quantize(DCT_subbands, block_y_side, block_x_side, N_components, Q_step)
DCT_subbands_dQ = block_DCT.uniform_dequantize(DCT_subbands_k, block_y_side, block_x_side, N_components, Q_step)
DCT_blocks_dQ = block_DCT.get_blocks(DCT_subbands_dQ, block_y_side, block_x_side)
YUV_img_dQ = block_DCT.synthesize_image(DCT_blocks_dQ, block_y_side, block_x_side)
img_dQ = color_transform.to_RGB(YUV_img_dQ) + 128
image.show(np.clip(img_dQ, a_min=0, a_max=255), f"Quantized image (Q_step={Q_step}) in the {block_y_side}x{block_x_side} DCT {color_transform.name} domain")

In [ ]:
error = img - img_dQ
image.show(image.normalize(error), "Quantization error")

Therefore, quantization in the DCT domain tends to remove high frequencies (it works basically as a low pass filter).

## Coding subbands vs coding blocks
Let's see the effect of encoding the DCT coefficients grouped by blocks and subbands. For simplicity, we will use uniform quantization.

In [ ]:
img = image.read(test_image, 0)
YUV_img = color_transform.from_RGB(img.astype(np.int16) - 128)

RD_points_blocks = []
RD_points_subbands = []
for Q_step in Q_steps:
    DCT_blocks = block_DCT.analyze_image(YUV_img, block_y_side, block_x_side)
    # Notice that with uniform_quantize() does not matter if the DCT domain
    # is organized in subbands or blocks.
    DCT_blocks_k = block_DCT.uniform_quantize(DCT_blocks, block_y_side, block_x_side, N_components, Q_step)
    BPP = image.write((DCT_blocks_k + 128).astype(np.uint8), f"/tmp/{Q_step}_", 0)*8/YUV_img.size
    # Check that we can recover the code-stream ################
    __ = image.read(f"/tmp/{Q_step}_", 0)                      #
    try:                                                       #
        assert ((DCT_blocks_k + 128) == __).all()              #
    except AssertionError:                                     #
        counter = 0                                            #
        for _i in range(img.shape[0]):                         #
            for _j in range(img.shape[1]):                     #
                if (DCT_blocks_k[_i, _j] != __[_i, _j]).any(): #
                    print(DCT_blocks_k[_i, _j], __[_i, _j])    #
                    if counter > 10:                           #
                        break                                  #
                    counter += 1                               #
            if counter > 10:                                   #
                break                                          #
    ############################################################
    DCT_blocks_dQ = block_DCT.uniform_dequantize(DCT_blocks_k, block_y_side, block_x_side, N_components, Q_step)
    YUV_img_dQ = block_DCT.synthesize_image(DCT_blocks_dQ, block_y_side, block_x_side)
    img_dQ = color_transform.to_RGB(YUV_img_dQ) + 128
    # Notice that to compute the distortion, the DCT domain could be
    # also used because the DCT is unitary.
    RMSE = distortion.RMSE(img, img_dQ)
    RD_points_blocks.append((BPP, RMSE))
    DCT_subbands_k = block_DCT.get_subbands(DCT_blocks_k, block_y_side, block_x_side)
    BPP = compute_BPP((DCT_subbands_k + 128).astype(np.uint8), f"/tmp/{Q_step}_")
    # Check that we can recover the code-stream #################
    __ = image.read(f"/tmp/{Q_step}_", 0)                       #
    try:                                                        #
        assert ((DCT_subbands_k + 128) == __).all()             #
    except AssertionError:                                      #
        counter = 0                                             #
        for _i in range(img.shape[0]):                          #
            for _j in range(img.shape[1]):                      #
                if (DCT_subbands_k[_i, _j] != __[_i, _j]).any():#
                    print(DCT_subbands_k[_i, _j], __[_i, _j])   #
                    if counter > 10:                            #
                        break                                   #
                    counter += 1                                #
            if counter > 10:                                    # 
                break                                           #
    #############################################################
    RD_points_subbands.append((BPP, RMSE))
    print(Q_step, end=' ', flush=True)

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points_blocks), label=f"{block_y_side}x{block_x_side} DCT (encoded by blocks)")
pylab.plot(*zip(*RD_points_subbands), label=f"{block_y_side}x{block_x_side} DCT (encoded by subbands)")
pylab.title("")
pylab.xlabel("BPP")
pylab.ylabel("RMSE")
plt.legend(loc="best")
pylab.show()

Coding by subbands is more efficient because PNG can exploit better the spatial correlation between the coefficients.

In [ ]:
with open('DCT_uniform.txt', 'w') as f:
    for item in RD_points_subbands:
        f.write(f"{item[0]}\t{item[1]}\n")

## Can we do it better?

Let's compute the optimal sequence of quantization steps for the set of possible combinations of subbands and components. We will compute the distortion of each subband-component in the YCoCg/8x8-DCT domain for a set of quantization steps, considering that the YCoCg transform is near-orthogonal and that the 8x8-DCT is full-orthogonal. Thanks to orthogonality, we can assume that the quantization error generated in one subband does not influence on the quantization error added to the other subbands because the DCT coefficients are uncorrelated, or in other words, that the quantization error generated in one coefficient (or subband) is not correlated with the quantization error generated in other coefficients (or subbands).

Algorithm:
1. Read the image.
2. Transform it to the YCoCg domain.
3. Transform each YCoCg component to the 8x8-DCT domain.
4. Find a set RD points for each subband-component.
5. Compute the slope of each point and put all the slopes in the same list.
6. Sort the previous list by the slope field.
7. Find the RD curve that progressively uses smaller slopes.

## Read the image and move to the YCoCg domain

In [ ]:
img = image.read(test_image, 0)
#xx = color_transform.from_RGB(img.astype(np.int16) - 128)
YUV_img = color_transform.from_RGB(img.astype(np.int16))

# Shift the YCoCg components to the zero mean. We will not need this information later (in this notebook)
# because we will not reconstruct the images. The distortion is computed in the DCT domain.
YUV_img[...,0] -= np.average(YUV_img[...,0]).astype(np.int16)
YUV_img[...,1] -= np.average(YUV_img[...,1]).astype(np.int16)
YUV_img[...,2] -= np.average(YUV_img[...,2]).astype(np.int16)

## Move each component to the 8x8-DCT domain

In [ ]:
DCT_blocks = block_DCT.analyze_image(YUV_img, block_y_side, block_x_side)
DCT_subbands = block_DCT.get_subbands(DCT_blocks, block_y_side, block_x_side)

In [ ]:
image.show(image.normalize(DCT_subbands), f"Subbands of the YCoCg/{block_y_side}x{block_x_side} DCT domain")

## Find the slope of each quantization step for each subband-component
Create a list per subband-component of RD points and a list per subband-component of RD slopes. The first RD point is computed for 0 BPP, where the MSE distortion is equal to the average energy of the subband-component (notice that the average of each subband-component should be 0).

In [ ]:
RD_points = []
RD_slopes = []
N_components = YUV_img.shape[2]
for _y in range(block_y_side):
    for _x in range(block_x_side):
        for _c in range(N_components):
            sbc = DCT_subbands[blocks_in_y*_y : blocks_in_y*(_y + 1), blocks_in_x*_x : blocks_in_x*(_x + 1), _c]
            sbc_energy = information.average_energy(sbc)
            # The first point of each RD curve has a maximum distortion equal
            # to the energy of the subband and a rate = 0
            RD_points.append([(0, sbc_energy)])
            RD_slopes.append([])

In [ ]:
RD_points # (BPP, RMSE)

In [ ]:
len(RD_points)

In [ ]:
8*8*3

Now populate the rest of points of each subband-component. **Distortion is estimated in the transform domain** supposing that both, the color and the spatial (DCT) transforms are orthogonal. Notice also that we consider only the length of the subband-component.

In [ ]:
for _y in range(block_y_side):
    for _x in range(block_x_side):
        for _c in range(N_components):
            sbc = DCT_subbands[blocks_in_y*_y : blocks_in_y*(_y + 1), blocks_in_x*_x : blocks_in_x*(_x + 1), _c]
            counter = 0
            for Q_step in Q_steps:
                sbc_k = Q.quantize(sbc, Q_step)
                sbc_dQ = Q.dequantize(sbc_k, Q_step)
                RMSE = distortion.RMSE(sbc, sbc_dQ)
                BPP = component.write(sbc_k.astype(np.uint8), f"/tmp/{_y}_{_x}_{_c}_{Q_step}_", 0)*8/YUV_img.size
                point = (BPP, RMSE)
                RD_points[(_y * block_x_side * N_components + _x * N_components ) + _c].append(point)
                print("Q_step =", Q_step, "BPP =", point[0], "RMSE =", point[1])
                delta_BPP = BPP - RD_points[(_y*block_x_side + _x)*N_components + _c][counter][0]
                delta_RMSE = RD_points[(_y*block_x_side + _x)*N_components + _c][counter][1] - RMSE
                if delta_BPP > 0:
                    slope = delta_RMSE/delta_BPP
                    RD_slopes[(_y*block_x_side + _x)*N_components + _c].append((slope, (_y, _x, _c), Q_step))
                else:
                    slope = 0
                #RD_slopes[(_y * block_x_side * N_components + _x * N_components) + _c].append((Q_step, slope, (_y, _x, _c)))
                counter += 1

In [ ]:
RD_points

In [ ]:
RD_slopes # (Qstep, slope, subband-component_index)

## Remove points that do not belong to the convex-hull

In [ ]:
def filter_slopes(slopes):
    filtered_slopes = []
    slopes_iterator = iter(slopes)
    prev = next(slopes_iterator)
    for curr in slopes_iterator:
        if prev[0] < curr[0]:
            print(f"deleted {prev}")
        else:
            filtered_slopes.append(prev)
        prev = curr
    filtered_slopes.append(prev)
    return filtered_slopes

filtered_slopes = []
for i in RD_slopes:
    filtered_slopes.append(filter_slopes(i))

In [ ]:
filtered_slopes

## Sort the RD points by their slope

In [ ]:
single_list = []
for l in filtered_slopes:
    #l = filter_slopes(l)
    for i in l:
        #if i[1] > 0:
        single_list.append(i)

In [ ]:
single_list

In [ ]:
sorted_slopes = sorted(single_list, key=lambda x: x[0])[::-1]

In [ ]:
sorted_slopes

## Build the optimal RD curve
We use the sorted list of slopes (with quantization and subband-component information) to generate the optimal RD list of RD points. Notice that, although the YCoCg components gains ($\frac{3}{2}\Delta_{\text{Y}} = \Delta_{\text{Co}} = \frac{3}{2}\Delta_{\text{Cg}}$) have not been taken into consideration, the RD points are sorted by their slope, and therefore this information has already influenced on the RD points.

Notice that initially, all the subband-components are initialized to zero, and the quantization steps (one per subband) are set to zero the subbands (see `Q_steps_combination`). Then, starting with the subband with the highest contribution (information that is provided by `sorted_slopes`) the, subband-components are progressively  quantized and dequantized.

Notice that the distortion can measured in the YCoCg/block-DCT domain because both transforms are considered orthogonal, making unnecessary the inverse transforms. Otherwise, the distortion should be measured in the image domain.

In [ ]:
optimal_RD_points = []
DCT_subbands_prog = np.zeros_like(DCT_subbands)
Q_steps_combination = np.full(shape=(block_x_side, block_y_side, N_components), fill_value=99999999)
for s in sorted_slopes:
    sbc_index = s[1]
    _y = sbc_index[0]
    _x = sbc_index[1]
    _c = sbc_index[2]
    Q_steps_combination[_y, _x, _c] = s[2]
    #DCT_subbands_prog[blocks_in_y*_y : blocks_in_y*(_y + 1), blocks_in_x*_x : blocks_in_x*(_x + 1), _c] \
    #    = DCT_subbands[blocks_in_y*_y : blocks_in_y*(_y + 1), blocks_in_x*_x : blocks_in_x*(_x + 1), _c]
    DCT_subbands_prog = DCT_subbands.copy()
    DCT_subbands_prog_k = block_DCT.quantize(DCT_subbands_prog, Q_steps_combination)
    DCT_subbands_prog_dQ = block_DCT.dequantize(DCT_subbands_prog_k, Q_steps_combination)
    
    # Uncomment the following line to measure the distortion in the color_transform+DCT domain
    RMSE = distortion.RMSE(DCT_subbands, DCT_subbands_prog_dQ)

    # Uncomment the following 3 lines to measure the distortion in the YUV domain
    #DCT_blocks_prog_dQ = DCT.get_blocks(DCT_subbands_prog_dQ, block_y_side, block_x_side)
    #YUV_img_prog = DCT.synthesize_image(DCT_blocks_prog_dQ, block_y_side, block_x_side)
    #RMSE = distortion.RMSE(YUV_img, YUV_img_prog)
    
    # If the color transform domain is not orthogonal, the RMSE should be measured in the RGB domain

    # Add 128 to convert 2's complement 8-bits integers to unsigned 8-bit integers.
    BPP = image.write((DCT_subbands_prog_k + 128).astype(np.uint8), f"/tmp/{_y}_{_x}_{_c}_{s[0]}_", 0)*8/YUV_img.size
    point = (BPP, RMSE)
    print("sbc =", sbc_index, "Q_step =", s[2], "BPP =", BPP, "RMSE =", RMSE)
    optimal_RD_points.append(point)

## Read JPEG RD data to compare
Notice that the chroma has not been subsampled.

In [ ]:
JPEG_RD_points = []
with open("JPEG.txt", 'r') as f:
    for line in f:
        rate, _distortion = line.split('\t')
        JPEG_RD_points.append((float(rate), float(_distortion)))

In [ ]:
#DCT2 = []
#with open("DCT.txt", 'r') as f:
#    for line in f:
#        rate, _distortion = line.split('\t')
#        DCT2.append((float(rate), float(_distortion)))

## Compare

In [ ]:
pylab.figure(dpi=150)
pylab.plot(*zip(*RD_points_subbands), label="Without RDO")
pylab.plot(*zip(*optimal_RD_points), label="With RDO")
#pylab.plot(*zip(*optimal_RD_points_128), label="optimal quantization 128")
pylab.plot(*zip(*JPEG_RD_points), label="JPEG")
#pylab.plot(*zip(*DCT2), label="old")
pylab.title("Comparing with JPEG")
pylab.xlabel("Bits/Pixel")
pylab.ylabel("RMSE")
plt.legend(loc="best")
#pylab.yscale('log')
#pylab.xscale('log')
pylab.show()

In [ ]:
with open('DCT_RDO.txt', 'w') as f:
    for item in optimal_RD_points:
        f.write(f"{item[0]}\t{item[1]}\n")

## Conclusions
The use of RD optimization increases the performance of the (color_transform + spatial_transform + entropy coding)-procedure, and more concretely of YCoCg + NxN-DCT + PNG. This is a consequence of selecting those quantization steps for the subband-components that contribute more to the quality of the reconstruction.